In [ ]:
!pip install -q transformers sentence-transformers faiss-cpu torch gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
!pip install torch

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 66.7 MB/s eta 0:00:00


In [ ]:
import sqlite3
import json
from datetime import datetime

def init_db(db_name="recipes.db"):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS recipes
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  name TEXT NOT NULL,
                  ingredients TEXT,
                  steps TEXT,
                  created_at TEXT)''')
    conn.commit()
    conn.close()

init_db()


In [ ]:
def import_json_to_db(json_file="dataset.json", db_name="recipes.db"):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()

    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            recipes = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"讀取 JSON 時發生錯誤：{e}")
        return

    if not isinstance(recipes, list):
        print("錯誤：recipes 應該是列表格式。")
        return

    for recipe in recipes:
        if not isinstance(recipe, dict) or "dish_name" not in recipe:
            continue

        ingredients_json = json.dumps(recipe.get("ingredients", []), ensure_ascii=False)
        steps_json = json.dumps(recipe.get("steps", []), ensure_ascii=False)
        c.execute(
            "INSERT INTO recipes (name, ingredients, steps, created_at) VALUES (?, ?, ?, ?)",
            (recipe["dish_name"], ingredients_json, steps_json, datetime.now().isoformat())
        )

    conn.commit()
    conn.close()
    print("資料成功匯入！")

import_json_to_db("dataset.json")


資料成功匯入！


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

def build_vector_store(db_name="recipes.db"):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute("SELECT name, ingredients, steps FROM recipes")
    results = c.fetchall()
    conn.close()

    embedding_model = SentenceTransformer('moka-ai/m3e-small')

    texts = []
    metadata = []
    for name, ingredients_json, steps_json in results:
        ingredients = json.loads(ingredients_json)
        steps = json.loads(steps_json)
        text = f"{name} {' '.join(ingredients)} {' '.join(steps)}"
        texts.append(text)
        metadata.append((name, ingredients, steps))

    embeddings = embedding_model.encode(texts, convert_to_numpy=True)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    return index, texts, metadata, embedding_model

index, texts, metadata, embedding_model = build_vector_store()


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/95.8M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/95.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def query_recipe_by_name(recipe_name, db_name="recipes.db"):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute("SELECT name, ingredients, steps FROM recipes WHERE name = ?", (recipe_name,))
    result = c.fetchone()
    conn.close()

    if result:
        name, ingredients_json, steps_json = result
        ingredients = json.loads(ingredients_json)
        steps = json.loads(steps_json)
        return [(name, ingredients, steps, "未知")]
    return None


In [ ]:
def search_recipes(query, index, texts, metadata, embedding_model, k=3):
    query_vector = embedding_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vector, k)
    results = []

    for idx in indices[0]:
        if idx >= 0 and idx < len(metadata):  # 確保索引合法
            name, ingredients, steps = metadata[idx]
            results.append((name, ingredients, steps, "未知"))

    return results


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

def generate_response(query, retrieved_recipes):
    if not retrieved_recipes:
        return "抱歉，找不到符合條件的簡單菜譜。"

    context = "以下是推薦的簡單菜譜：\n"
    for name, ingredients, steps, cuisine in retrieved_recipes[:1]:
        context += f"菜名: {name}\n食材: {', '.join(ingredients)}\n步驟: {', '.join(steps)}\n"

    prompt = f"使用者問：{query}\n{context}\n請用簡潔的中文回應，推薦這道菜並說明做法，不要添加無關內容。"

    response = generator(prompt, max_new_tokens=200, num_return_sequences=1, truncation=True)[0]['generated_text']
    generated_text = response[len(prompt):].strip()

    if len(generated_text) < 10:
        name, ingredients, steps, _ = retrieved_recipes[0]
        return f"推薦「{name}」。\n食材：{', '.join(ingredients)}\n做法：{', '.join(steps)}"

    return generated_text


Device set to use cuda:0


In [ ]:
def rag_pipeline(query, index, texts, metadata, embedding_model):
    if not query:
        return "請輸入查詢！"

    query = query.strip()

    # 查詢並檢索食譜
    retrieved_recipes = search_recipes(query, index, texts, metadata, embedding_model)

    # 若無法找到食譜，返回相應訊息
    if not retrieved_recipes:
        return "抱歉，找不到符合條件的簡單菜譜。請試著更改您的查詢。"

    return generate_response(f"請推薦與{query}相關的簡單料理", retrieved_recipes)


In [ ]:
import gradio as gr

# 定義 Gradio 介面
interface = gr.Interface(
    fn=lambda query: rag_pipeline(query, index, texts, metadata, embedding_model),
    inputs=gr.Textbox(label="輸入查詢（例如：三杯雞 或 請推薦一道用雞肉做的簡單料理）"),
    outputs=gr.Textbox(label="食譜推薦"),
    title="簡單食譜推薦系統",
    description="輸入菜名查詢特定食譜，或輸入食材/描述尋找相關料理！"
)

# 啟動 Gradio 介面
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ebd32265e2f595064.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
